In [1]:
#This performs Hierarchical modeling on a partially-observed birth-death process 0->X->0 with birth parameter A and death parameter B.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import gamma
from scipy.stats import poisson
import math

In [2]:
#import saved data
pd_subsampled_x=pd.read_csv("subsampled.csv")
x= pd_subsampled_x.values
size=np.shape(x)

In [3]:
L=3000      #length of the chain
I=size[0]    #I-1 is number of subintervals
M=size[1]    #number of individuals
#initializations
A=np.zeros((L,M))               #birth parameter
B=np.zeros((L,M))               #death parameter
a_A=np.zeros(L+1)               #birth shape hyperparameter
a_B=np.zeros(L+1)               #death shape hyperparameter
b_A=np.zeros(L+1)               #birth scale hyperparameter
b_B=np.zeros(L+1)               #death scale hyperparameter
r=np.zeros((2,I-1,size[1]))     #reaction numbers
sum_x=np.zeros(M)   
var_a_A=2                       #birth shape hyperparameter proposal variance
var_a_B=2                       #death shape hyperparameter proposal variance
#initial conditions
a_A[0]=25
b_A[0]=0.1
a_B[0]=25
b_B[0]=0.1

In [4]:
def init_reaction(x,s):         #initialize reaction numbers           
    q=np.zeros([2,s-1])
    for i in range(s-1):
        if x[i+1]-x[i]>0:
            q[0,i]=np.floor((4/3)*(x[i+1]-x[i]))
            q[1,i]=np.floor((1/3)*(x[i+1]-x[i]))
        else:
            q[0,i]=-np.floor((1/3)*(x[i+1]-x[i]))
            q[1,i]=-np.floor((4/3)*(x[i+1]-x[i]))
    r=q.astype(int)        
    return r  

def accept_rate_react(x,r,prop,i,A,B):        #acceptance rate for reaction numbers
    prop_like=poisson.pmf(prop[0],A)*poisson.pmf(prop[1],0.5*B*(x[i]+x[i+1]))
    current_like=poisson.pmf(r[0],A)*poisson.pmf(r[1],0.5*B*(x[i]+x[i+1]))  
    rate=np.minimum(1,prop_like/current_like)
    return rate

def a_lik(A,a,b):           #shape hyperparameter likelihood function 
    sum1=0
    M=len(A)
    for s in range(M):
        sum1+=np.log(A[s])
    sum1*=(a-1)
    q=-M*np.log(math.gamma(a))+M*a*np.log(b)+sum1
    return q

In [5]:
#initialize number of reactions
for s in range(M):
    r[:,:,s]= init_reaction(x[:,s],I)
    sum_x[s]=x[0,s]+x[I-1,s]
    for j in range(I-2):
        sum_x[s]+=2*x[j,s]
    sum_x[s]=0.5*sum_x[s] 

In [ ]:
r=np.load('react.npy')

In [6]:
#perform sampling       
for i in range(L):
    for s in range(M):
        #sample parameters
        A[i,s]=np.random.gamma(np.sum(r[0,:,s])+a_A[i],(I-1+b_A[i])**(-1))
        B[i,s]=np.random.gamma(np.sum(r[1,:,s])+a_B[i],(sum_x[s]+b_A[i])**(-1))

        #update reaction numbers
        for j in range(I-1):
            r_vprop=np.array([-1,-1])
            while r_vprop[0]<0 or r_vprop[1]<0:
                r_prop=r[0,j,s]+np.random.poisson(1)-np.random.poisson(1)
                r_vprop=[r_prop, r_prop-(x[j+1,s]-x[j,s])]    
            rate=accept_rate_react(x[:,s],r[:,j,s],r_vprop,j,A[i,s],B[i,s]) 
            if np.random.uniform(0,1,1)<rate:
                r[:,j,s]=r_vprop      
    
    #sample hyperparameters a_A and a_B
    a_Aprop=-1
    while a_Aprop<0:
        a_Aprop=a_A[i]+np.random.normal(0,var_a_A)
    a=a_lik(A[i,:],a_Aprop,b_A[i]) 
    b=a_lik(A[i,:],a_A[i],b_A[i])
    rate=np.minimum(1,np.exp(a-b))
    if np.random.uniform(0,1,1)<rate:    
        a_A[i+1]=a_Aprop
    else:
        a_A[i+1]=a_A[i]
        
    a_Bprop=-1
    while a_Bprop<0:
        a_Bprop=a_B[i]+np.random.normal(0,var_a_B)
    a=a_lik(B[i,:],a_Bprop,b_B[i]) 
    b=a_lik(B[i,:],a_B[i],b_B[i])
    rate=np.minimum(1,np.exp(a-b))
    if np.random.uniform(0,1,1)<rate:    
        a_B[i+1]=a_Bprop
    else:
        a_B[i+1]=a_B[i]    
        
    #sample hyperparameters b_A and b_B
    sum1=np.sum(A[i,:])
    sum2=np.sum(B[i,:])
    b_A[i+1]=np.random.gamma(M*a_A[i+1],sum1**(-1))
    b_B[i+1]=np.random.gamma(M*a_B[i+1],sum2**(-1))
    
    if i%20 == 0:
        print(a_A[i+1],b_A[i+1],"next",b_A[i+1],b_B[i+1])                        

21.627089437107074 8.712789684652083 next 8.712789684652083 208.1203118234744
22.776947530816454 6.1131000306293775 next 6.1131000306293775 121.21882479060783
21.721856577563223 5.2994809720714855 next 5.2994809720714855 84.34759532469297
28.353996587722474 6.0396258600453825 next 6.0396258600453825 80.87577648889992
25.987116572090915 5.339408078809599 next 5.339408078809599 79.31663298744658
23.81564160241115 5.064323928971318 next 5.064323928971318 52.18218787164065
25.02185987779358 5.322357607741612 next 5.322357607741612 45.81894385993202
26.483965408965286 5.622464219471414 next 5.622464219471414 50.03609836143755
29.899707534394924 6.707799683886254 next 6.707799683886254 61.369769683640925
29.616391437721774 6.160996484962505 next 6.160996484962505 58.510819792075196
29.547824619127958 6.097351404376399 next 6.097351404376399 63.29411367445752
27.01759003402694 5.895095920428828 next 5.895095920428828 64.75023199571787
29.319214222556404 6.44641748133272 next 6.44641748133272 

KeyboardInterrupt: 

In [ ]:
#save the simulated values into .csv files
np.save('react.npy',r) 
columns=' '
np.savetxt("A.csv", A, delimiter=",",header=columns)
np.savetxt("B.csv", B, delimiter=",",header=columns)
np.savetxt("a_A.csv", a_A, delimiter=",",header=columns)
np.savetxt("b_A.csv", b_A, delimiter=",",header=columns)
np.savetxt("a_B.csv", a_B, delimiter=",",header=columns)
np.savetxt("b_B.csv", b_B, delimiter=",",header=columns)